In [3]:
%matplotlib inline 
import matplotlib.pylab
import pandas as pd
import numpy as np

In [4]:
with open('data/ao_monthly.txt') as f:
    for x in range(5):
        print(next(f))

 1950    1  -0.60310E-01

 1950    2   0.62681E+00

 1950    3  -0.81275E-02

 1950    4   0.55510E+00

 1950    5   0.71577E-01



In [5]:
data = pd.read_fwf('data/ao_monthly.txt', header = None)

In [6]:
# Not so great
data.head()

,0,1,2
0,1950,1,-0.060310
1,1950,2,0.626810
2,1950,3,-0.008127
3,1950,4,0.555100
4,1950,5,0.071577


### Look at the options for read_fwf...what looks relevant?

In [11]:
# %load snippets/readtime.py
data = pd.read_fwf('data/ao_monthly.txt', header = None,parse_dates=[[0,1]])

In [12]:
data.head()

,0_1,2
0,1950-01-01,-0.060310
1,1950-02-01,0.626810
2,1950-03-01,-0.008127
3,1950-04-01,0.555100
4,1950-05-01,0.071577


In [13]:
data.index

RangeIndex(start=0, stop=798, step=1)

In [15]:
data.index.names = ['Month']
data.columns = ['Value']
data.head()

ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements

### What is the empirical range of dates?

In [1]:
# %load snippets/daterange.py
min(data.index)

NameError: name 'data' is not defined

### How can we convert to complementary representation?

In [31]:
# %load snippets/changerep.py

### More about reading in data

When reading in dates with a pd.read function, you have several time-related parameters you can adjust: 
parse_dates, infer_datetime_format, date_parser

Experiment with these using %timeit to see if there are performance differences

Hint:
infer_datetime_format = True, no date parser provided
What other combos can you come up with?

In [41]:
import timeit
# First, let's see how to use a date_parser:
dateparse = lambda x, y: pd.datetime.strptime('%s-%s'%(x,y), '%Y-%m')

%timeit data = pd.read_fwf('ao_monthly.txt', header = None, index_col = 0, parse_dates = [[0, 1]], date_parser = dateparse)

100 loops, best of 3: 14 ms per loop


### You can also extract datetimes from existing Data Frame columns

In [3]:
# new in pandas 0.18
df = pd.DataFrame({'year':[2015, 2016], 'month':[2,3], 'day':[4,5], 'hour':[12, 13]})
df

,day,hour,month,year
0,4,12,2,2015
1,5,13,3,2016


In [4]:
pd.to_datetime(df)

0   2015-02-04 12:00:00
1   2016-03-05 13:00:00
dtype: datetime64[ns]

### Truncating

In [33]:
ts = pd.Series(range(10), index = pd.date_range('7/31/15', freq = 'M', periods = 10))
ts

2015-07-31    0
2015-08-31    1
2015-09-30    2
2015-10-31    3
2015-11-30    4
2015-12-31    5
2016-01-31    6
2016-02-29    7
2016-03-31    8
2016-04-30    9
Freq: M, dtype: int64

In [34]:
# truncating preserves frequency
ts.truncate(before = '10/31/2015', after = '12/31/2015')

2015-10-31    3
2015-11-30    4
2015-12-31    5
Freq: M, dtype: int64

In [35]:
# You can truncate in a way that does not preserve frequency
ts[[1, 6, 7]].index

DatetimeIndex(['2015-08-31', '2016-01-31', '2016-02-29'], dtype='datetime64[ns]', freq=None)

In [37]:
# But Pandas will try to preserve frequency automatically whenever possible
ts[0:10:2]

2015-07-31    0
2015-09-30    2
2015-11-30    4
2016-01-31    6
2016-03-31    8
Freq: 2M, dtype: int64